In [44]:
import psycopg
import os
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

# Paramètres de connexion
db_params = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

# Test de connexion
try:
    conn = psycopg.connect(**db_params)
    print("✅ Connexion réussie à PostgreSQL!")
    print(f"Version PostgreSQL: {conn.info.server_version}")
    conn.close()
except Exception as e:
    print(f"❌ Erreur de connexion: {e}")

✅ Connexion réussie à PostgreSQL!
Version PostgreSQL: 170007


In [45]:
import psycopg
from pgvector.psycopg import register_vector
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import os
from dotenv import load_dotenv
import numpy as np

print("✅ Imports réussis!")

✅ Imports réussis!


In [46]:
# Charger variables d'environnement
load_dotenv()

# Configuration PostgreSQL
DB_CONFIG = {
    'dbname': os.getenv('DB_NAME', 'rag_chatbot'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5433')
}

print("✅ Configuration chargée!")
print(f"Base: {DB_CONFIG['dbname']} sur port {DB_CONFIG['port']}")

✅ Configuration chargée!
Base: rag_chatbot sur port 5433


In [47]:
# Connexion
try:
    conn = psycopg.connect(**DB_CONFIG)
    register_vector(conn)
    print("✅ Connexion PostgreSQL réussie!")
    print(f"Version: {conn.info.server_version}")
except Exception as e:
    print(f"❌ Erreur: {e}")

✅ Connexion PostgreSQL réussie!
Version: 170007


In [48]:
# Vérifier que pgvector est activé
conn.rollback()

with conn.cursor() as cur:
    cur.execute("""
        SELECT name, default_version
        FROM pg_available_extensions
        WHERE name = 'vector';
    """)
    row = cur.fetchone()

if row:
    print(f"✅ pgvector disponible (version par défaut : {row[1]})")
else:
    print("❌ pgvector NON disponible")


✅ pgvector disponible (version par défaut : 0.8.1)


In [49]:
conn.rollback()

with conn.cursor() as cur:
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    conn.commit()
    print("✅ pgvector activé dans la base")


✅ pgvector activé dans la base


In [50]:
#Charger le modèle d’embeddings
from sentence_transformers import SentenceTransformer

print("📥 Chargement du modèle d'embedding...")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
print("✅ Modèle chargé!")

# Test rapide
test_emb = model.encode("Test")
print(f"Dimension embedding: {len(test_emb)}")


📥 Chargement du modèle d'embedding...
✅ Modèle chargé!
Dimension embedding: 768


In [51]:
# Créer la table pour stocker les embeddings
conn.rollback()

with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS embeddings (
            id SERIAL PRIMARY KEY,
            corpus TEXT NOT NULL,
            embedding VECTOR(768)  -- Dimension du modèle all-mpnet-base-v2
        )
    """)
    conn.commit()
    print("✅ Table 'embeddings' créée!")

# Vérifier le nombre d'entrées
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM embeddings")
    count = cur.fetchone()[0]
    print(f"📊 Nombre d'entrées actuelles : {count}")

✅ Table 'embeddings' créée!
📊 Nombre d'entrées actuelles : 0


In [52]:
import glob

# Chemin vers les fichiers du corpus
CORPUS_PATH = "../data/TRANS_TXT/*.txt"

# Lister les fichiers
fichiers = glob.glob(CORPUS_PATH)

if not fichiers:
    print("⚠️ ATTENTION : Aucun fichier trouvé!")
    print(f"📁 Vérifie que le corpus est dans : {CORPUS_PATH}")
else:
    print(f"✅ {len(fichiers)} fichiers trouvés!")
    print(f"📄 Premiers fichiers :")
    for f in fichiers[:5]:
        print(f"  - {os.path.basename(f)}")

✅ 41 fichiers trouvés!
📄 Premiers fichiers :
  - 017_00000012.txt
  - 018_00000013.txt
  - 019_00000014.txt
  - 020_00000015.txt
  - 022_00000017.txt


In [53]:
import time

print("🚀 Début du chargement du corpus...\n")

documents_inseres = 0

for i, filepath in enumerate(fichiers, 1):
    try:
        # Lire le fichier
        with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()
        
        # Ignorer les fichiers trop courts
        if len(text.strip()) < 50:
            print(f"⏭️ [{i}/{len(fichiers)}] Fichier ignoré (trop court)")
            continue
        
        # Générer l'embedding avec sentence-transformers
        embedding = model.encode(text, convert_to_numpy=True)
        embedding_list = embedding.tolist()
        
        # Insérer dans la base
        with conn.cursor() as cur:
            cur.execute("""
                INSERT INTO embeddings (corpus, embedding)
                VALUES (%s, %s)
            """, (text, embedding_list))
        
        conn.commit()
        documents_inseres += 1
        
        print(f"✅ [{i}/{len(fichiers)}] {os.path.basename(filepath)} - {len(text)} caractères")
        
        # Pause pour ne pas surcharger
        time.sleep(0.3)
        
    except Exception as e:
        print(f"❌ Erreur avec {filepath}: {e}")
        conn.rollback()
        continue

print(f"\n✅ Chargement terminé! {documents_inseres} documents insérés.")

# Vérifier le total
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM embeddings")
    total = cur.fetchone()[0]
    print(f"📊 Total d'entrées dans la base : {total}")

🚀 Début du chargement du corpus...

✅ [1/41] 017_00000012.txt - 2539 caractères
✅ [2/41] 018_00000013.txt - 455 caractères
✅ [3/41] 019_00000014.txt - 748 caractères
✅ [4/41] 020_00000015.txt - 578 caractères
✅ [5/41] 022_00000017.txt - 1187 caractères
✅ [6/41] 023_00000018.txt - 786 caractères
✅ [7/41] 024_00000019.txt - 5276 caractères
✅ [8/41] 027_0000001c.txt - 2175 caractères
✅ [9/41] 028_0000001d.txt - 5172 caractères
✅ [10/41] 029_0000001e.txt - 442 caractères
✅ [11/41] 030_0000001f.txt - 335 caractères
✅ [12/41] 037_00000026.txt - 764 caractères
✅ [13/41] 038_00000027.txt - 1681 caractères
✅ [14/41] 045_0000002e.txt - 423 caractères
✅ [15/41] 047_00000030.txt - 2542 caractères
✅ [16/41] 048_00000031.txt - 1615 caractères
✅ [17/41] 052_00000035.txt - 353 caractères
✅ [18/41] 053_00000036.txt - 1032 caractères
✅ [19/41] 054_00000037.txt - 1861 caractères
✅ [20/41] 055_00000038.txt - 445 caractères
✅ [21/41] 057_0000003a.txt - 2194 caractères
✅ [22/41] 058_0000003b.txt - 648 carac

In [54]:
from pgvector.psycopg import Vector

def search_similar(query_text, top_k=5):
    """
    Recherche les documents les plus similaires à la requête.
    
    Args:
        query_text (str): Texte de la requête
        top_k (int): Nombre de résultats à retourner
        
    Returns:
        list: Liste de tuples (id, corpus, distance)
    """
    # Générer l'embedding de la requête
    query_embedding = model.encode(query_text, convert_to_numpy=True)
    query_vector = Vector(query_embedding.tolist())
    
    # Rechercher les documents similaires
    with conn.cursor() as cur:
        cur.execute("""
            SELECT id, corpus, embedding <-> %s AS distance
            FROM embeddings
            ORDER BY distance
            LIMIT %s
        """, (query_vector, top_k))
        return cur.fetchall()

print("✅ Fonction de recherche définie!")

✅ Fonction de recherche définie!


In [55]:
# Test de recherche
query = "Bonjour, je cherche des informations"

print(f"🔎 Requête : {query}\n")

results = search_similar(query, top_k=3)

print(f"📊 {len(results)} résultats trouvés :\n")

for i, (doc_id, corpus, distance) in enumerate(results, 1):
    print(f"{'='*80}")
    print(f"🏆 Résultat {i}")
    print(f"📍 ID: {doc_id} | 📏 Distance: {distance:.4f}")
    print(f"📄 Extrait:\n{corpus[:250]}...\n")

🔎 Requête : Bonjour, je cherche des informations

📊 3 résultats trouvés :

🏆 Résultat 1
📍 ID: 24 | 📏 Distance: 0.9505
📄 Extrait:
<01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour madame j'aurais voulu avoir des renseignements pour e l'() l'inscription en A E S administration conomique et sociale
<03> hotesse
     h: Nom oui [pi] oui conservez je vais vous pass...

🏆 Résultat 2
📍 ID: 25 | 📏 Distance: 0.9638
📄 Extrait:
<01> hotesse
     h: U B S bonjour
<02> client
     c: bonjour ici Prnom Nom est ce que je pourrais avoir Prnom2 Nom2 s'il vous plait
<03> hotesse
     h: # oui
...

🏆 Résultat 3
📍 ID: 35 | 📏 Distance: 0.9762
📄 Extrait:
<01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour e j'aurais voulu savoir je postule pour e donc j'tais en licence d'conomie  Rennes #  Vannes pardon [pf] cette anne
<03> hotesse
     h: oui
<04> client
     c: et donc je postule pou...



In [57]:
# Créer la table pour stocker les embeddings
conn.rollback()

with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS embeddings (
            id SERIAL PRIMARY KEY,
            corpus TEXT NOT NULL,
            embedding VECTOR(768)
        )
    """)
    conn.commit()
    print("✅ Table 'embeddings' créée!")

# Vérifier
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM embeddings")
    count = cur.fetchone()[0]
    print(f"📊 Nombre d'entrées actuelles : {count}")

✅ Table 'embeddings' créée!
📊 Nombre d'entrées actuelles : 40


In [58]:
import glob

CORPUS_PATH = "../data/TRANS_TXT/*.txt"
fichiers = glob.glob(CORPUS_PATH)

if not fichiers:
    print("⚠️ ATTENTION : Aucun fichier trouvé!")
else:
    print(f"✅ {len(fichiers)} fichiers trouvés!")
    for f in fichiers[:5]:
        print(f"  - {os.path.basename(f)}")
```

**→ Exécute cette cellule et dis-moi combien de fichiers sont trouvés !**

---

## 🚨 **PROBLÈME POTENTIEL : Chemin du Corpus**

Dans ton `.env`, tu as :
```
DB_PORT=5433
```

Mais est-ce que ton corpus est bien dans :
```
C:\Users\hp\CHATBOT-RAG\data\TRANS_TXT\

SyntaxError: invalid character '→' (U+2192) (91635150.py, line 14)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
